In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp feature_engineering

# Feature engineering
> Create exogenous regressors for your models

In [ ]:
#| export
from functools import partial
from typing import Callable, List, Optional, Tuple, Union

import numpy as np
import pandas as pd

import utilsforecast.processing as ufp
from utilsforecast.compat import DFType, DataFrame, pl, pl_DataFrame, pl_Expr
from utilsforecast.validation import validate_format, validate_freq

In [ ]:
#| exporti
_Features = Tuple[List[str], np.ndarray, np.ndarray]

def _add_features(
    df: DFType,
    freq: Union[str, int],
    h: int,
    id_col: str,
    time_col: str,
    f: Callable[[np.ndarray, int], _Features],
) -> Tuple[DFType, DFType]:
    # validations
    if not isinstance(h, int) or h < 0:
        raise ValueError('`h` must be a non-negative integer')
    validate_format(df, id_col, time_col, None)
    validate_freq(df[time_col], freq)

    # decompose series
    id_counts = ufp.counts_by_id(df, id_col)
    uids = id_counts[id_col]
    sizes = id_counts['counts'].to_numpy()

    # compute values
    cols, vals, future_vals = f(sizes=sizes, h=h)  # type: ignore

    # assign back to df
    sort_idxs = ufp.maybe_compute_sort_indices(df, id_col, time_col)
    times = df[time_col]
    if sort_idxs is not None:
        restore_idxs = np.empty_like(sort_idxs)
        restore_idxs[sort_idxs] = np.arange(sort_idxs.size)
        vals = vals[restore_idxs]
        times = ufp.take_rows(times, sort_idxs)
    last_times = ufp.take_rows(times, sizes.cumsum() - 1)
    df = ufp.copy_if_pandas(df, deep=False)
    transformed = ufp.assign_columns(df, cols, vals)

    if h == 0:
        return transformed, type(df)({})

    # future vals
    future_df = ufp.make_future_dataframe(
        uids=uids,
        last_times=last_times,
        freq=freq,
        h=h,
        id_col=id_col,
        time_col=time_col,
    )
    future_df = ufp.assign_columns(future_df, cols, future_vals)
    return transformed, future_df

def _assign_slices(
    sizes: np.ndarray,
    feats: np.ndarray,
    h: int,
) -> Tuple[np.ndarray, np.ndarray]:
    max_samples, n_feats = feats.shape
    vals = np.empty((sizes.sum(), n_feats), dtype=np.float32)
    future_vals = np.empty((h * sizes.size, n_feats))
    start = 0
    for i, size in enumerate(sizes):
        vals[start : start + size, :] = feats[max_samples - size - h : max_samples - h]
        future_vals[i * h: (i + 1) * h] = feats[max_samples - h :]
        start += size
    return vals, future_vals

def _fourier(
    sizes: np.ndarray,
    h: int,
    season_length: int,
    k: int,
) -> _Features:
    # taken from: https://github.com/tblume1992/TSUtilities/blob/main/TSUtilities/TSFeatures/fourier_seasonality.py
    x = 2 * np.pi * np.arange(1, k + 1) / season_length
    x = x.astype(np.float32)
    t = np.arange(1, sizes.max() + 1 + h, dtype=np.float32)
    x = x * t[:, None]
    terms = np.hstack([np.sin(x), np.cos(x)])
    cols = [f'{op}{i+1}_{season_length}' for op in ('sin', 'cos') for i in range(k)]
    vals, future_vals = _assign_slices(sizes=sizes, feats=terms, h=h)
    return cols, vals, future_vals

def _trend(sizes: np.ndarray, h: int) -> _Features:
    t = np.arange(1, sizes.max() + 1 + h, dtype=np.float32).reshape(-1, 1)
    cols = ['trend']
    vals, future_vals = _assign_slices(sizes=sizes, feats=t, h=h)
    return cols, vals, future_vals

In [ ]:
#| export
def fourier(
    df: DFType,
    freq: Union[str, int],
    season_length: int,
    k: int,
    h: int = 0,
    id_col: str = 'unique_id',
    time_col: str = 'ds',
) -> Tuple[DFType, DFType]:
    """Compute fourier seasonal terms for training and forecasting

    Parameters
    ----------
    df : pandas or polars DataFrame
        Dataframe with ids, times and values for the exogenous regressors.
    freq : str or int
        Frequency of the data. Must be a valid pandas or polars offset alias, or an integer.
    season_length : int
        Number of observations per unit of time. Ex: 24 Hourly data.
    k : int
        Maximum order of the fourier terms
    h : int (default=0)
        Forecast horizon.        
    id_col : str (default='unique_id')
        Column that identifies each serie.
    time_col : str (default='ds')
        Column that identifies each timestep, its values can be timestamps or integers.

    Returns
    -------
    transformed_df : pandas or polars DataFrame
        Original DataFrame with the computed features
    future_df : pandas or polars DataFrame
        DataFrame with future values
    """
    f = partial(_fourier, season_length=season_length, k=k)
    return _add_features(
        df=df,
        freq=freq,
        h=h,
        id_col=id_col,
        time_col=time_col,
        f=f,
    )

In [ ]:
import pandas as pd

from utilsforecast.data import generate_series

In [ ]:
series = generate_series(5, equal_ends=True)
transformed_df, future_df = fourier(series, freq='D', season_length=7, k=2, h=1)
transformed_df

,unique_id,ds,y,sin1_7,sin2_7,cos1_7,cos2_7
0,0,2000-10-05,0.428973,-0.974927,0.433894,-0.222526,-0.900964
1,0,2000-10-06,1.423626,-0.781835,-0.974926,0.623486,-0.222531
2,0,2000-10-07,2.311782,-0.000005,-0.000009,1.000000,1.000000
3,0,2000-10-08,3.192191,0.781829,0.974930,0.623493,-0.222512
4,0,2000-10-09,4.148767,0.974929,-0.433877,-0.222517,-0.900972
...,...,...,...,...,...,...,...
1096,4,2001-05-10,4.058910,-0.974927,0.433888,-0.222523,-0.900967
1097,4,2001-05-11,5.178157,-0.781823,-0.974934,0.623500,-0.222495
1098,4,2001-05-12,6.133142,-0.000002,-0.000003,1.000000,1.000000
1099,4,2001-05-13,0.403709,0.781840,0.974922,0.623479,-0.222548


In [ ]:
future_df

,unique_id,ds,sin1_7,sin2_7,cos1_7,cos2_7
0,0,2001-05-15,0.433871,-0.781813,-0.900975,0.623513
1,1,2001-05-15,0.433871,-0.781813,-0.900975,0.623513
2,2,2001-05-15,0.433871,-0.781813,-0.900975,0.623513
3,3,2001-05-15,0.433871,-0.781813,-0.900975,0.623513
4,4,2001-05-15,0.433871,-0.781813,-0.900975,0.623513


In [ ]:
#| hide
transformed_df2, future_df2 = fourier(series.sample(frac=1.0), freq='D', season_length=7, k=2, h=1)
pd.testing.assert_frame_equal(
    transformed_df,
    transformed_df2.sort_values(['unique_id', 'ds']).reset_index(drop=True)
)
pd.testing.assert_frame_equal(future_df, future_df2)

In [ ]:
#| hide
#| polars
import polars as pl
import polars.testing

In [ ]:
#| hide
#| polars
series_pl = generate_series(5, equal_ends=True, engine='polars')
transformed_pl, future_pl = fourier(series_pl, freq='1d', season_length=7, k=2, h=1)
transformed_pl2, future_pl2 = fourier(series_pl.sample(fraction=1.0), freq='1d', season_length=7, k=2, h=1)
pl.testing.assert_frame_equal(transformed_pl, transformed_pl2)
pl.testing.assert_frame_equal(future_pl, future_pl2)
pd.testing.assert_frame_equal(
    transformed_df.drop(columns=['unique_id']),
    transformed_pl.drop('unique_id').to_pandas()
)
pd.testing.assert_frame_equal(
    future_df.drop(columns=['unique_id']),
    future_pl.drop('unique_id').to_pandas()
)

In [ ]:
#| export
def trend(
    df: DFType,
    freq: Union[str, int],
    h: int = 0,
    id_col: str = 'unique_id',
    time_col: str = 'ds',
) -> Tuple[DFType, DFType]:
    """Add a trend column with consecutive integers for training and forecasting

    Parameters
    ----------
    df : pandas or polars DataFrame
        Dataframe with ids, times and values for the exogenous regressors.
    freq : str or int
        Frequency of the data. Must be a valid pandas or polars offset alias, or an integer.
    h : int (default=0)
        Forecast horizon.        
    id_col : str (default='unique_id')
        Column that identifies each serie.
    time_col : str (default='ds')
        Column that identifies each timestep, its values can be timestamps or integers.

    Returns
    -------
    transformed_df : pandas or polars DataFrame
        Original DataFrame with the computed features
    future_df : pandas or polars DataFrame
        DataFrame with future values
    """
    return _add_features(
        df=df,
        freq=freq,
        h=h,
        id_col=id_col,
        time_col=time_col,
        f=_trend,
    )

In [ ]:
series = generate_series(5, equal_ends=True)
transformed_df, future_df = trend(series, freq='D', h=1)
transformed_df

,unique_id,ds,y,trend
0,0,2000-10-05,0.428973,152.0
1,0,2000-10-06,1.423626,153.0
2,0,2000-10-07,2.311782,154.0
3,0,2000-10-08,3.192191,155.0
4,0,2000-10-09,4.148767,156.0
...,...,...,...,...
1096,4,2001-05-10,4.058910,369.0
1097,4,2001-05-11,5.178157,370.0
1098,4,2001-05-12,6.133142,371.0
1099,4,2001-05-13,0.403709,372.0


In [ ]:
future_df

,unique_id,ds,trend
0,0,2001-05-15,374.0
1,1,2001-05-15,374.0
2,2,2001-05-15,374.0
3,3,2001-05-15,374.0
4,4,2001-05-15,374.0


In [ ]:
#| exporti
def _compute_time_feature(
    times: Union[pd.Index, pl_Expr],
    feature: Union[str, Callable],
) -> Tuple[Union[str, List[str]], Union[pd.DataFrame, pl_Expr, List[pl_Expr], pd.Index, np.ndarray]]:
    if callable(feature):
        feat_vals = feature(times)
        if isinstance(feat_vals, pd.DataFrame):
            feat_name = feat_vals.columns.tolist()
            feat_vals = feat_vals.to_numpy()
        else:
            feat_name = feature.__name__
    else:
        feat_name = feature
        if isinstance(times, pd.DatetimeIndex):
            if feature in ("week", "weekofyear"):
                times = times.isocalendar()
            feat_vals = getattr(times, feature).to_numpy()
        else:
            feat_vals = getattr(times.dt, feature)()
    return feat_name, feat_vals

def _add_time_features(
    df: DFType,
    features: List[Union[str, Callable]],
    time_col: str = 'ds',
) -> DFType:
    df = ufp.copy_if_pandas(df, deep=False)
    unique_times = df[time_col].unique()
    if isinstance(df, pd.DataFrame):
        times = pd.Index(unique_times)
        time2pos = {time: i for i, time in enumerate(times)}
        restore_idxs = df[time_col].map(time2pos).to_numpy()
        for feature in features:
            name, vals = _compute_time_feature(times, feature)
            df[name] = vals[restore_idxs]
    elif isinstance(df, pl_DataFrame):
        exprs = []
        for feature in features:
            name, vals = _compute_time_feature(pl.col(time_col), feature)
            if isinstance(vals, list):
                exprs.extend(vals)
            else:
                assert isinstance(vals, pl_Expr)
                exprs.append(vals.alias(name))
        feats = unique_times.to_frame().with_columns(*exprs)
        df = df.join(feats, on=time_col, how='left')
    return df

In [ ]:
#| export
def time_features(
    df: DFType,
    freq: Union[str, int],
    features: List[Union[str, Callable]],
    h: int = 0,
    id_col: str = 'unique_id',
    time_col: str = 'ds',    
) -> Tuple[DFType, DFType]:
    """Compute timestamp-based features for training and forecasting

    Parameters
    ----------
    df : pandas or polars DataFrame
        Dataframe with ids, times and values for the exogenous regressors.
    freq : str or int
        Frequency of the data. Must be a valid pandas or polars offset alias, or an integer.
    features : list of str or callable
        Features to compute. Can be string aliases of timestamp attributes or functions to apply to the times.
    h : int (default=0)
        Forecast horizon.
    id_col : str (default='unique_id')
        Column that identifies each serie.
    time_col : str (default='ds')
        Column that identifies each timestep, its values can be timestamps or integers.

    Returns
    -------
    transformed_df : pandas or polars DataFrame
        Original DataFrame with the computed features
    future_df : pandas or polars DataFrame
        DataFrame with future values
    """
    transformed = _add_time_features(df=df, features=features, time_col=time_col)
    if h == 0:
        return transformed, type(df)({})
    times_by_id = ufp.group_by_agg(df, id_col, {time_col: 'max'}, maintain_order=True)
    times_by_id = ufp.sort(times_by_id, id_col)
    future = ufp.make_future_dataframe(
        uids=times_by_id[id_col],
        last_times=times_by_id[time_col],
        freq=freq,
        h=h,
        id_col=id_col,
        time_col=time_col,
    )
    future = _add_time_features(df=future, features=features, time_col=time_col)
    return transformed, future

In [ ]:
transformed_df, future_df = time_features(series, freq='D', features=['month', 'day', 'week'], h=1)
transformed_df

,unique_id,ds,y,month,day,week
0,0,2000-10-05,0.428973,10,5,40
1,0,2000-10-06,1.423626,10,6,40
2,0,2000-10-07,2.311782,10,7,40
3,0,2000-10-08,3.192191,10,8,40
4,0,2000-10-09,4.148767,10,9,41
...,...,...,...,...,...,...
1096,4,2001-05-10,4.058910,5,10,19
1097,4,2001-05-11,5.178157,5,11,19
1098,4,2001-05-12,6.133142,5,12,19
1099,4,2001-05-13,0.403709,5,13,19


In [ ]:
future_df

,unique_id,ds,month,day,week
0,0,2001-05-15,5,15,20
1,1,2001-05-15,5,15,20
2,2,2001-05-15,5,15,20
3,3,2001-05-15,5,15,20
4,4,2001-05-15,5,15,20


In [ ]:
#| export
def future_exog_to_historic(
    df: DFType,
    freq: Union[str, int],
    features: List[str],
    h: int = 0,
    id_col: str = 'unique_id',
    time_col: str = 'ds',    
) -> Tuple[DFType, DFType]:
    """Turn future exogenous features into historic by shifting them `h` steps.

    Parameters
    ----------
    df : pandas or polars DataFrame
        Dataframe with ids, times and values for the exogenous regressors.
    freq : str or int
        Frequency of the data. Must be a valid pandas or polars offset alias, or an integer.
    features : list of str
        Features to be converted into historic.
    h : int (default=0)
        Forecast horizon.
    id_col : str (default='unique_id')
        Column that identifies each serie.
    time_col : str (default='ds')
        Column that identifies each timestep, its values can be timestamps or integers.

    Returns
    -------
    transformed_df : pandas or polars DataFrame
        Original DataFrame with the computed features
    future_df : pandas or polars DataFrame
        DataFrame with future values
    """
    if h == 0:
        return df, type(df)({})
    new_feats = ufp.copy_if_pandas(df[[id_col, time_col, *features]])
    new_feats = ufp.assign_columns(
        new_feats,
        time_col,
        ufp.offset_times(new_feats[time_col], freq=freq, n=h),
    )
    df = ufp.drop_columns(df, features)
    df = ufp.join(df, new_feats, on=[id_col, time_col], how='left')
    times_by_id = ufp.group_by_agg(df, id_col, {time_col: 'max'}, maintain_order=True)
    times_by_id = ufp.sort(times_by_id, id_col)
    future = ufp.make_future_dataframe(
        uids=times_by_id[id_col],
        last_times=times_by_id[time_col],
        freq=freq,
        h=h,
        id_col=id_col,
        time_col=time_col,
    )
    future = ufp.join(future, new_feats, on=[id_col, time_col], how='left')
    return df, future

In [ ]:
series_with_prices = series.assign(price=np.random.rand(len(series))).sample(frac=1.0)
series_with_prices

,unique_id,ds,y,price
436,2,2001-03-26,2.369113,0.774476
312,1,2001-05-08,4.405212,0.557957
536,3,2000-11-04,4.362074,0.745237
34,0,2000-11-08,6.111161,0.809978
652,3,2001-02-28,1.448291,0.685294
...,...,...,...,...
609,3,2001-01-16,0.215892,0.699703
873,4,2000-09-29,5.398198,0.677651
268,1,2001-03-25,2.393771,0.735438
171,0,2001-03-25,3.085493,0.463871


In [ ]:
transformed_df, future_df = future_exog_to_historic(
    df=series_with_prices, 
    freq='D',
    features=['price'],
    h=2,
)
transformed_df

,unique_id,ds,y,price
0,2,2001-03-26,2.369113,0.870133
1,1,2001-05-08,4.405212,0.869751
2,3,2000-11-04,4.362074,0.877901
3,0,2000-11-08,6.111161,0.629413
4,3,2001-02-28,1.448291,0.088073
...,...,...,...,...
1096,3,2001-01-16,0.215892,0.472261
1097,4,2000-09-29,5.398198,0.887531
1098,1,2001-03-25,2.393771,0.481712
1099,0,2001-03-25,3.085493,0.433153


In [ ]:
future_df

,unique_id,ds,price
0,0,2001-05-15,0.874328
1,0,2001-05-16,0.481385
2,1,2001-05-15,0.009058
3,1,2001-05-16,0.083749
4,2,2001-05-15,0.726212
5,2,2001-05-16,0.052221
6,3,2001-05-15,0.942335
7,3,2001-05-16,0.274816
8,4,2001-05-15,0.267545
9,4,2001-05-16,0.112129


In [ ]:
#| hide
pd.testing.assert_frame_equal(
    (
        series_with_prices
        .sort_values(['unique_id', 'ds'])
        .groupby('unique_id', observed=True)
        .tail(2)
        [['unique_id', 'price']]
        .reset_index(drop=True)
    ),
    future_df[['unique_id', 'price']]
)

In [ ]:
#| hide
#| polars
series_with_prices_pl = pl.from_pandas(
    series_with_prices.astype({'unique_id': 'int64'})
)
transformed_pl, future_pl = future_exog_to_historic(
    df=series_with_prices_pl, 
    freq='1d',
    features=['price'],
    h=2,
)
pd.testing.assert_frame_equal(
    future_pl.to_pandas(),
    future_df.astype({'unique_id': 'int64'})
)

In [ ]:
#| export
def pipeline(
    df: DFType,
    features: List[Callable],
    freq: Union[str, int],
    h: int = 0,
    id_col: str = 'unique_id',
    time_col: str = 'ds',
) -> Tuple[DFType, DFType]:
    """Compute several features for training and forecasting

    Parameters
    ----------
    df : pandas or polars DataFrame
        Dataframe with ids, times and values for the exogenous regressors.
    features : list of callable
        List of features to compute. Must take only df, freq, h, id_col and time_col (other arguments must be fixed).
    freq : str or int
        Frequency of the data. Must be a valid pandas or polars offset alias, or an integer.
    h : int (default=0)
        Forecast horizon.        
    id_col : str (default='unique_id')
        Column that identifies each serie.
    time_col : str (default='ds')
        Column that identifies each timestep, its values can be timestamps or integers.

    Returns
    -------
    transformed_df : pandas or polars DataFrame
        Original DataFrame with the computed features
    future_df : pandas or polars DataFrame
        DataFrame with future values
    """
    transformed: Optional[DataFrame] = None
    future: Optional[DataFrame] = None
    for f in features:
        f_transformed, f_future = f(df=df, freq=freq, h=h, id_col=id_col, time_col=time_col)
        if transformed is None:
            transformed = f_transformed
            future = f_future
        else:
            feat_cols = [c for c in f_future.columns if c not in (id_col, time_col)]
            transformed = ufp.horizontal_concat([transformed, f_transformed[feat_cols]])
            future = ufp.horizontal_concat([future, f_future[feat_cols]])
    return transformed, future

In [ ]:
def is_weekend(times):
    if isinstance(times, pd.Index):
        dow = times.weekday + 1  # monday=0 in pandas and 1 in polars
    else:
        dow = times.dt.weekday()
    return dow >= 6

def even_days_and_months(times):
    if isinstance(times, pd.Index):
        out = pd.DataFrame(
            {
                'even_day': (times.weekday + 1) % 2 == 0,
                'even_month': times.month % 2 == 0,
            }
        )
    else:
        # for polars you can return a list of expressions
        out = [
            (times.dt.weekday() % 2 == 0).alias('even_day'),
            (times.dt.month() % 2 == 0).alias('even_month'),
        ]
    return out

features = [
    trend,
    partial(fourier, season_length=7, k=1),
    partial(fourier, season_length=28, k=1),
    partial(time_features, features=['day', is_weekend, even_days_and_months]),
]
transformed_df, future_df = pipeline(
    series,
    features=features,
    freq='D',
    h=1,
)
transformed_df

,unique_id,ds,y,trend,sin1_7,cos1_7,sin1_28,cos1_28,day,is_weekend,even_day,even_month
0,0,2000-10-05,0.428973,152.0,-0.974927,-0.222526,0.433885,-9.009683e-01,5,False,True,True
1,0,2000-10-06,1.423626,153.0,-0.781835,0.623486,0.222522,-9.749276e-01,6,False,False,True
2,0,2000-10-07,2.311782,154.0,-0.000005,1.000000,0.000001,-1.000000e+00,7,True,True,True
3,0,2000-10-08,3.192191,155.0,0.781829,0.623493,-0.222520,-9.749281e-01,8,True,False,True
4,0,2000-10-09,4.148767,156.0,0.974929,-0.222517,-0.433883,-9.009693e-01,9,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...
1096,4,2001-05-10,4.058910,369.0,-0.974927,-0.222523,0.900969,4.338843e-01,10,False,True,False
1097,4,2001-05-11,5.178157,370.0,-0.781823,0.623500,0.974929,2.225177e-01,11,False,False,False
1098,4,2001-05-12,6.133142,371.0,-0.000002,1.000000,1.000000,4.251100e-07,12,True,True,False
1099,4,2001-05-13,0.403709,372.0,0.781840,0.623479,0.974927,-2.225243e-01,13,True,False,False


In [ ]:
future_df

,unique_id,ds,trend,sin1_7,cos1_7,sin1_28,cos1_28,day,is_weekend,even_day,even_month
0,0,2001-05-15,374.0,0.433871,-0.900975,0.781829,-0.623493,15,False,True,False
1,1,2001-05-15,374.0,0.433871,-0.900975,0.781829,-0.623493,15,False,True,False
2,2,2001-05-15,374.0,0.433871,-0.900975,0.781829,-0.623493,15,False,True,False
3,3,2001-05-15,374.0,0.433871,-0.900975,0.781829,-0.623493,15,False,True,False
4,4,2001-05-15,374.0,0.433871,-0.900975,0.781829,-0.623493,15,False,True,False


In [ ]:
#| hide
from functools import reduce

In [ ]:
#| hide
def reduce_join(dfs, on):
    return reduce(
        lambda left, right: left.merge(right, on=on, how='left'),
        dfs,
    )

individual_results = [f(series, freq='D', h=1) for f in features]
expected_transformed = reduce_join(
    [r[0] for r in individual_results], on=['unique_id', 'ds', 'y']
)
expected_future = reduce_join(
    [r[1] for r in individual_results], on=['unique_id', 'ds']
)
pd.testing.assert_frame_equal(transformed_df, expected_transformed)
pd.testing.assert_frame_equal(future_df, expected_future)

In [ ]:
#| hide
#| polars
transformed_pl, future_pl = pipeline(
    series_pl,
    features=features,
    freq='1d',
    h=1,
)
pd.testing.assert_frame_equal(
    transformed_pl.drop('unique_id').to_pandas(),
    transformed_df.drop(columns='unique_id'),
    check_dtype=False,
)
pd.testing.assert_frame_equal(
    future_pl.drop('unique_id').to_pandas(),
    future_df.drop(columns='unique_id'),
    check_dtype=False,
)